<a href="https://colab.research.google.com/github/9645258/aiffel/blob/main/02_goingdeeper/01_nlp/%5BGD-16%5Dnlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GD NLP 02 : HuggingFace 커스텀 프로젝트 만들기**

#### **Train/Evaluation과 Test**

- **MNLI 데이터셋을 처리하는 전용 Processor 클래스를 정상적으로 구현하였다.**  
Processor 클래스에 대해 1개 이상의 example에 대한 단위테스트가 정상 진행되었다.  

- **BERT tokenizer와 Processor를 결합하여 데이터셋을 정상적으로 생성하였다.**  
MNLI 데이터셋의 입력과 라벨의 정의에 잘 맞는 tf.data.Dataset 인스턴스가 얻어졌다.  

- **MNLI 데이터셋에 대해 적당한 모델을 fine-tuning하여 학습하였다.**  
모델 학습이 정상적으로 진행되었다.


## **1. 데이터 준비**

### **1-1. 라이브러리 import**

In [ ]:
import os
from dataclasses import asdict

import warnings
warnings.filterwarnings(action = 'ignore')

import tensorflow as tf
import tensorflow_datasets as tfds

import transformers
transformers.logging.set_verbosity_error()

from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification

### **1-2. 데이터 load**

In [ ]:
dataset, info = tfds.load('glue/mnli', with_info=True)

### **1-3. 데이터 확인**

- **데이터 분류 및 각 데이터 분류에 포함된 데이터 수 확인**  
train / validation_matched / validation_mismatched / test_matched / test_mismatched 총 5개의 데이터 분류  

In [ ]:
for idx, (domain, data) in enumerate(dataset.items()):
    print(f"{idx}: {domain}({len(data):,})")

0: train(392,702)
1: validation_matched(9,815)
2: validation_mismatched(9,832)
3: test_matched(9,796)
4: test_mismatched(9,847)


- **train 데이터 확인**

In [ ]:
dataset['train']

<PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [ ]:
for data in dataset['train']:
    print("Hypothesis:", data['hypothesis'].numpy(), end="\n")
    print("idx:", data['idx'].numpy(), end="\n")    
    print("Premise:", data['premise'].numpy(), end="\n")
    print("Label:", data['label'].numpy())    
    break

Hypothesis: b'Meaningful partnerships with stakeholders is crucial.'
idx: 16399
Premise: b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'
Label: 1


- **validation_matched 데이터 확인**

In [ ]:
dataset['validation_matched']

<PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [ ]:
for data in dataset['validation_matched']:
    print("Hypothesis:", data['hypothesis'].numpy(), end="\n")
    print("idx:", data['idx'].numpy(), end="\n")    
    print("Premise:", data['premise'].numpy(), end="\n")
    print("Label:", data['label'].numpy())
    break

Hypothesis: b'yeah lots of people for the right life '
idx: 6287
Premise: b'uh-huh oh yeah all the people for right uh life or something'
Label: 0


- **validation_mismatched 데이터 확인**

In [ ]:
dataset['validation_mismatched']

<PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [ ]:
for data in dataset['validation_mismatched']:
    print("Hypothesis:", data['hypothesis'].numpy(), end="\n")
    print("idx:", data['idx'].numpy(), end="\n")    
    print("Premise:", data['premise'].numpy(), end="\n")
    print("Label:", data['label'].numpy())
    break

Hypothesis: b"These projects are largely ignored and don't impact anyone. "
idx: 9410
Premise: b'Projects which enliven and enrich the student experience and draw some of our finest scholars and teachers to our campus--and to our city.'
Label: 2


- **test_matched 데이터 확인**

In [ ]:
dataset['test_matched']

<PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [ ]:
for data in dataset['test_matched']:
    print("Hypothesis:", data['hypothesis'].numpy(), end="\n")
    print("idx:", data['idx'].numpy(), end="\n")    
    print("Premise:", data['premise'].numpy(), end="\n")
    print("Label:", data['label'].numpy())
    break

Hypothesis: b'Is there a pump that you put in there?'
idx: 5398
Premise: b'well is there a little electric pump you put in there'
Label: -1


- **test_mismatched 데이터 확인**

In [ ]:
dataset['test_mismatched']

<PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [ ]:
for data in dataset['test_mismatched']:
    print("Hypothesis:", data['hypothesis'].numpy(), end="\n")
    print("idx:", data['idx'].numpy(), end="\n")    
    print("Premise:", data['premise'].numpy(), end="\n")
    print("Label:", data['label'].numpy())
    break

Hypothesis: b'That table has been in my family for generations.'
idx: 3498
Premise: b'Anyway, I treasure that table.'
Label: -1


## **2. 데이터 전처리**

### **2-1. 데이터셋 split**

- **데이터셋 split 함수 정의**

In [ ]:
def split_dataset(dataset, val_size):    
    dataset = dataset.shuffle(len(dataset))
    val_dataset = dataset.shuffle(len(dataset)).take(val_size)
    test_dataset = dataset.shuffle(len(dataset)).skip(val_size)
    return (val_dataset, test_dataset)

- **data 데이터셋 결합**

In [ ]:
matched_data = dataset['test_matched'].concatenate(dataset['validation_matched'])
mismatched_data = dataset['test_mismatched'].concatenate(dataset['validation_mismatched'])

- **데이터셋 split 진행**

In [ ]:
matched_val, matched_test = split_dataset(matched_data, len(dataset['validation_matched']))
mismatched_val, mismatched_test = split_dataset(mismatched_data, len(dataset['validation_mismatched']))

- **val 데이터셋 결합**

In [ ]:
val = mismatched_val.concatenate(matched_val)

- **데이터 수 확인**

In [ ]:
print('학습 데이터에 포함된 validation 데이터 수:', len(matched_val))
print('학습 데이터에 포함되지 않은 validation 데이터 수:', len(mismatched_val))
print('학습 데이터에 포함된 test 데이터 수:', len(matched_test))
print('학습 데이터에 포함되지 않은 test 데이터 수:', len(mismatched_test))

학습 데이터에 포함된 validation 데이터 수: 9815
학습 데이터에 포함되지 않은 validation 데이터 수: 9832
학습 데이터에 포함된 test 데이터 수: 9796
학습 데이터에 포함되지 않은 test 데이터 수: 9847


### **2-2. MNLI Processor**

- **추상 클래스 생성**

In [ ]:
class DataProcessor:

    def get_example_from_tensor_dict(self, tensor_dict):
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        raise NotImplementedError()

    def get_labels(self):
        raise NotImplementedError()

    def tfds_map(self, example):
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

- **추상 클래스 상속**

In [ ]:
class MnliProcessor(DataProcessor):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_example_from_tensor_dict(self, tensor_dict):
        return InputExample(tensor_dict["idx"].numpy(), tensor_dict["premise"].numpy().decode("utf-8"), 
                            tensor_dict["hypothesis"].numpy().decode("utf-8"), str(tensor_dict["label"].numpy()), )

    def get_train_examples(self, data_dir):
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev_matched.tsv")), "dev_matched")

    def get_test_examples(self, data_dir):
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test_matched.tsv")), "test_matched")

    def get_labels(self):
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, lines, set_type):
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            text_b = line[4]
            label = None if set_type == "test" else line[0]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

- **MNLI Processor을 이용한 전처리 예시 확인**

In [ ]:
processor = MnliProcessor()
examples = dataset['train'].take(1)

In [ ]:
for example in examples:
    
    print("<원본 데이터>", type(example), sep = "\n")
    print(example, "\n\n")
    
    example = processor.get_example_from_tensor_dict(example)
    
    print("<Processed 데이터>", type(example), sep = "\n")
    print(example, "\n\n")

    label_map = {label: i for i, label in enumerate(processor.get_labels())}
    print("<라벨>")
    print(label_map)

<원본 데이터>
<class 'dict'>
{'hypothesis': <tf.Tensor: shape=(), dtype=string, numpy=b'Meaningful partnerships with stakeholders is crucial.'>, 'idx': <tf.Tensor: shape=(), dtype=int32, numpy=16399>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'premise': <tf.Tensor: shape=(), dtype=string, numpy=b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'>} 


<Processed 데이터>
<class 'transformers.data.processors.utils.InputExample'>
InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services

## **3. 데이터셋 구성**

### **3-1. 정수 인코딩**

- **데이터 정수화 함수 정의**

In [ ]:
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    
    if max_length is None :
        max_length = tokenizer.max_len
        
    if label_list is None:
        label_list = processor.get_labels()

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer([(example.text_a, example.text_b) for example in examples], 
                               max_length=max_length, padding="max_length", truncation=True, )

    features = []
    
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    return features

### **3-2. 데이터셋 생성**

- **정수화 진행 데이터 데이터셋화 함수 정의**

In [ ]:
def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :

    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(gen, ({k: tf.int32 for k in input_names}, label_type),
                                          ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])), )

### **3-3. 토크나이저**

- **토크나이저 import**

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base", use_fast=True)

### **3-4. 데이터셋 구성**

- **학습 데이터셋 생성**

In [ ]:
def get_dataset_batch(raw_dataset_list, tokenizer, processor):
    
    dataset_list = []
    
    for idx, dataset in enumerate(raw_dataset_list):
        data = tf_glue_convert_examples_to_features(dataset, tokenizer, max_length=128, processor=processor)
        
        if idx == 0:
            data_batch = data.shuffle(100).batch(16).repeat(2)
        else:
            data_batch = data.shuffle(100).batch(16)
        dataset_list.append(data_batch)
    return dataset_list

In [ ]:
raw_dataset_list = (dataset['train'], val, matched_test, mismatched_test)
bert_train, bert_val, bert_m_test, bert_mism_test = get_dataset_batch(raw_dataset_list, bert_tokenizer, processor)
robert_train, robert_val, robert_m_test, robert_mism_test = get_dataset_batch(raw_dataset_list, roberta_tokenizer, processor)

## **4. 모델 생성 및 학습 진행**

### **4-1. 옵티마이저**

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

### **4-2. BERT**

- **모델 생성**

In [ ]:
bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
bert_model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
bert_model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________


- **모델 학습**

In [ ]:
bert_model.fit(bert_train, epochs=20, steps_per_epoch=500, validation_data=bert_val)

Epoch 1/20
500/500 [==============================] - 119s 217ms/step - loss: 0.8550 - acc: 0.6111 - val_loss: 1.0582 - val_acc: 0.5197
Epoch 2/20
500/500 [==============================] - 108s 217ms/step - loss: 0.7099 - acc: 0.7056 - val_loss: 1.2419 - val_acc: 0.5454
Epoch 3/20
500/500 [==============================] - 108s 216ms/step - loss: 0.6380 - acc: 0.7425 - val_loss: 1.1351 - val_acc: 0.5632
Epoch 4/20
500/500 [==============================] - 106s 213ms/step - loss: 0.6089 - acc: 0.7505 - val_loss: 1.2829 - val_acc: 0.5590
Epoch 5/20
500/500 [==============================] - 107s 214ms/step - loss: 0.6086 - acc: 0.7571 - val_loss: 1.1551 - val_acc: 0.5468
Epoch 6/20
500/500 [==============================] - 107s 214ms/step - loss: 0.5802 - acc: 0.7691 - val_loss: 1.4161 - val_acc: 0.5321
Epoch 7/20
500/500 [==============================] - 107s 215ms/step - loss: 0.5744 - acc: 0.7673 - val_loss: 1.2186 - val_acc: 0.5653
Epoch 8/20
500/500 [============================

### **4-3. RoBERTa**

- **모델 생성**

In [ ]:
roberta_model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)
roberta_model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
roberta_model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  124055040 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  592899    
Total params: 124,647,939
Trainable params: 124,647,939
Non-trainable params: 0
_________________________________________________________________


- **모델 학습**

In [ ]:
roberta_model.fit(robert_train, epochs=20, steps_per_epoch=500, validation_data=robert_val)

Epoch 1/20
500/500 [==============================] - 115s 212ms/step - loss: 1.1049 - acc: 0.3246 - val_loss: 1.1445 - val_acc: 0.1767
Epoch 2/20
500/500 [==============================] - 104s 209ms/step - loss: 1.1007 - acc: 0.3302 - val_loss: 1.1129 - val_acc: 0.1767
Epoch 3/20
500/500 [==============================] - 105s 210ms/step - loss: 1.1010 - acc: 0.3306 - val_loss: 1.1171 - val_acc: 0.1767
Epoch 4/20
500/500 [==============================] - 105s 209ms/step - loss: 1.1003 - acc: 0.3391 - val_loss: 1.1339 - val_acc: 0.1589
Epoch 5/20
500/500 [==============================] - 105s 211ms/step - loss: 1.1001 - acc: 0.3269 - val_loss: 1.0862 - val_acc: 0.6645
Epoch 6/20
500/500 [==============================] - 105s 211ms/step - loss: 1.0997 - acc: 0.3414 - val_loss: 1.0739 - val_acc: 0.6645
Epoch 7/20
500/500 [==============================] - 105s 211ms/step - loss: 1.1004 - acc: 0.3298 - val_loss: 1.0914 - val_acc: 0.1767
Epoch 8/20
500/500 [============================

## **5. 모델 평가**

### **5-1. BERT**

In [ ]:
bert_matched_result = bert_model.evaluate(bert_m_test)
bert_mismatched_result = bert_model.evaluate(bert_mism_test)

616/616 [==============================] - 27s 43ms/step - loss: 1.1076 - acc: 0.5962


In [ ]:
print(f"<학습 데이터에 포함된 test 데이터>\nLoss: {bert_matched_result[0]:.3f}\nAccuracy: {bert_matched_result[1]:.3f}\n\n")
print(f"<학습 데이터에 포함되지 않은 test 데이터>\nLoss: {bert_mismatched_result[0]:.3f}\nAccuracy: {bert_mismatched_result[1]:.3f}")

<학습 데이터에 포함된 test 데이터>
Loss: 1.117
Accuracy: 0.599


<학습 데이터에 포함되지 않은 test 데이터>
Loss: 1.108
Accuracy: 0.596


### **5-2. RoBERTa**

In [ ]:
roberta_matched_result = roberta_model.evaluate(robert_m_test)
roberta_mismatched_result = roberta_model.evaluate(robert_mism_test)

616/616 [==============================] - 26s 42ms/step - loss: 1.0624 - acc: 0.6642


In [ ]:
print(f"<학습 데이터에 포함된 test 데이터>\nLoss: {roberta_matched_result[0]:.3f}\nAccuracy: {roberta_matched_result[1]:.3f}\n\n")
print(f"<학습 데이터에 포함되지 않은 test 데이터>\nLoss: {roberta_mismatched_result[0]:.3f}\nAccuracy: {roberta_mismatched_result[1]:.3f}")

<학습 데이터에 포함된 test 데이터>
Loss: 1.062
Accuracy: 0.665


<학습 데이터에 포함되지 않은 test 데이터>
Loss: 1.062
Accuracy: 0.664


### **5-3. 최종 평가**

||Validation 데이터|학습 데이터에 포함된 Test 데이터|학습 데이터에 포함되지 않은 Test 데이터|
|:---:|:---:|:---:|:---:|
||**Loss / Accuracy**|**Loss / Accuracy**|**Loss / Accuracy**|
|**BERT**|1.1041 / 0.6016|1.117 / 0.599|1.108 / 0.596|
|**RoBERTa**|1.0624 / 0.6645|1.062 / 0.665|1.062 / 0.664|

## **6. 회고**

### **6-1. 프로젝트 회고**

이번 프로젝트는 16번 노드 내용을 LMS상에서 진행했을 떄 굉장히 많은 오류를 만났기 때문에 잘 진행할 수 있는지 의문이 들었다. 이번 프로젝트는 로컬에서 진행을 했는데, 버전 호환 문제사 발생할 것이라고 생각하고 먼저 다른 사람의 코드를 똑같이 진행해 보았다. 그랬는데 생각보다 오류 한번 없이 잘 진행되었다! 그래서 생각보다 굉장히 수월하게 진행할 수 있었다.  

중간에 학습데이터를 생성하는 과정에서 오류는 발생하지 않았지만 자꾸 경고 문구를 출력하는데, 데이터 한줄을 생성할 때 마다 한번씩 출력되어서 경고 문구를 삭제하는 라이브러리를 불러와서 사용했다. 아무래도 프레임워크를 가져와서 사용하는 것이다 보니 버전 문제만 해결되면 라이브러리를 불러와서 이런 저런 수정을 편리하게 할 수 있는 부분이 아주 매력적이고, 잘 배워 놓으면 잘 사용할 수 있다는 생각이 들었다.  

일단 다른 분의 코드를 똑같이 적는 방식으로 진행을 하였지만, 저번까지 내가 모델을 구성했던 것에 비해 Hugging Face에서 제공하는 모델을 가져다 썼기 때문에 더 간단하게 작업이 가능한 점이 좋았던 것 같다. 대부분의 프레임워크가 파이토치를 주로 지원하는 점 때문에 지금까지는 tensorflow를 주로 사용했지만 파이토치도 사용해야 할 필요성이 느껴졌다. 시간이 난다면 파이토치를 조금씩 공부를 해야할 것 같다.  

### **6-2. 참고 자료**

1. https://github.com/YAGI0423/aiffel_going_deeper_nlp/blob/main/going_deeper_16/GD16_v3_1.ipynb